In [1]:
%load_ext autoreload
%autoreload 2
# TODO: Use this to profile: https://stackoverflow.com/questions/45893768/how-do-i-find-out-what-parts-of-my-code-are-inefficient-in-python
%load_ext line_profiler

In [14]:
import matplotlib.pyplot as plt
import pandas as pd
import holidays
import dataclasses
import datetime
import pprint
import statistics

from src.serialization_lib import *
from src.data_types import *
from datetime import timedelta
from enum import Enum, auto
from typing import List, Optional, Tuple, Set
from pandas.tseries.offsets import BDay
pd.set_option('display.max_colwidth', None)

pp = pprint.PrettyPrinter(indent=4)

In [3]:
API_KEY = 'cfg2wsKZrVNuYBJpETAs'
DEVELOPMENT = False

if DEVELOPMENT:
    daily_metrics = pd.read_csv('SHARADAR-DAILY.csv')
    daily_prices = pd.read_csv('SHARADAR-SEP.csv')
else:
    daily_metrics = pd.read_csv('SHARADAR_DAILY_3_9ffd00fad4f19bbdec75c6e670d3df83.csv')
    daily_prices = pd.read_csv('SHARADAR_SEP_2_0bd2000858d1d8d1f48d4cdea5f8c9e2.csv')

In [4]:
d1 = daily_metrics.copy()

d2 = daily_prices[['ticker', 'date','closeadj']]
d2.rename(columns={'closeadj': 'price'}, inplace=True)

daily_data = d1.merge(d2, on=['date', 'ticker'], how='inner')
daily_data

/Users/olshansky/.pyenv/versions/3.7.7/envs/tip_env/lib/python3.7/site-packages/pandas/core/frame.py:4449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,ticker,date,lastupdated,ev,evebit,evebitda,marketcap,pb,pe,ps,price
0,A,2020-12-18,2020-12-18,37525.1,40.8,30.6,36607.1,7.5,50.9,6.9,118.928
1,AA,2020-12-18,2020-12-18,4896.3,-43.0,8.9,4092.3,1.2,-8.7,0.4,22.010
2,AACQ,2020-12-18,2020-12-18,943.3,NaN,NaN,944.6,188.9,NaN,NaN,10.430
3,AAIC,2020-12-18,2020-12-18,817.6,-15.8,-15.8,125.0,0.5,-2.3,5.3,3.740
4,AAIIQ,2020-12-18,2020-12-18,-2.8,0.5,0.8,0.4,0.0,-0.1,0.0,0.100
...,...,...,...,...,...,...,...,...,...,...,...
14551055,GSTCQ,2011-01-04,2021-05-30,235.6,3.4,3.0,218.6,1.4,-575.3,5.0,4.340
14551056,MHRCQ,2011-01-04,2021-05-30,471.6,-29.7,-64.3,419.2,6.0,-19.3,13.6,7.230
14551057,EPRSQ,2011-01-03,2021-05-30,153.1,187.2,8.6,155.8,2.1,74.4,2.9,105.000
14551058,GSTCQ,2011-01-03,2021-05-30,242.7,3.5,3.1,225.7,1.4,-593.9,5.1,4.480


In [5]:
def get_closest_previous_work_day(
    check_day: datetime.datetime,
    holidays=holidays.US()
) -> datetime.datetime:
    if check_day.weekday() <= 4 and check_day not in holidays:
        return check_day
    offset = max(1, (check_day.weekday() + 6) % 7 - 3)
    most_recent = check_day - datetime.timedelta(offset)
    if most_recent not in holidays:
        return most_recent
    else:
        return get_closest_previous_work_day(most_recent, holidays)

def get_rebalance_dates(
    start_date: datetime.datetime,
    end_date: datetime.datetime,
    period_length: datetime.timedelta
) -> List[datetime.datetime]:
    curr_date = start_date
    dates = []
    while curr_date < end_date:
        dates.append(get_closest_previous_work_day(curr_date))
        curr_date += period_length
    return dates

def sort_df_by_metric(
    df: pd.DataFrame,
    metric: EvaluationMetric
) -> pd.DataFrame:
    if metric.value == EvaluationMetric.EV_EBIT.value:
        return df.sort_values(by='evebit')
    elif metric.value == EvaluationMetric.P_E.value:
        return df.sort_values(by='pe')
    elif metric.value == EvaluationMetric.P_B.value:
        return df.sort_values(by='pb')
    elif metric.value == EvaluationMetric.DIV_YIELD.value:
        raise Exception('EvaluationMetric.DIV_YIELD not yet supported.')
    else:
        raise Exception(f'Unsupported evaluation metric {metric}')

def filter_df_by_date(
    df: pd.DataFrame,
    date: datetime.datetime
) -> pd.DataFrame:
    return df[df.date == date.strftime('%Y-%m-%d')]        
        
# ASSUMPTION: df is already filtered by date.
def filter_stocks_by_universe(
    df: pd.DataFrame,
    stocks_universe: StockUniverse
) -> pd.DataFrame:
    if stocks_universe.value == StockUniverse.SMALL.value:
        return df[df['marketcap'] < 1]
    elif stocks_universe.value == StockUniverse.MID.value:
        return df[(df['marketcap'] >= 1) & (df['marketcap'] <= 10)]
    elif stocks_universe.value == StockUniverse.LARGE.value:
        return df[(df['marketcap'] >= 10)]
    else:
        raise Exception(f'Unsupported stock universe {stocks_universe}')

# ASSUMPTION: df is sorted by metric.
def get_top_n_stocks_by_metric(
    df: pd.DataFrame,
    n: int,
    metric: EvaluationMetric    
) -> List[str]:
    df_res = None
    if metric.value == EvaluationMetric.EV_EBIT.value:
        df_res = df[(df['evebit'] > 0) & (df['ev'] > 0)]
    elif metric.value == EvaluationMetric.P_E.value:
        df_res = df[df['pe'] > 0]
    elif metric.value == EvaluationMetric.P_B.value:
        df_res = df[df['pb'] > 0]
    elif metric.value == EvaluationMetric.DIV_YIELD.value:
        raise Exception('EvaluationMetric.DIV_YIELD not yet supported.')
    else:
        raise Exception(f'Unsupported evaluation metric {metric}')
    
    return list(df_res[:n]['ticker'])

# ASSUMPTION: df is sorted by date.
def get_last_available_price(
    df: pd.DataFrame,
    ticker: str
) -> int:
    return df[df.ticker == ticker].iloc[-1]['price']

# ASSUMPTION: df is filtered by date.
def get_stock_price_per_stock(
    df: pd.DataFrame,
    df_prev: Optional[pd.DataFrame],    
    df_full: pd.DataFrame,  # Used to get the prices for stocks that are not available (acquired or closed)
    tickers: List[str]
) -> Tuple[str, int]:
    res = []
    stocks_of_interest = df.loc[df['ticker'].isin(tickers)]
    missing_stocks = set(tickers) - set(stocks_of_interest['ticker'])
    
    for idx, r in stocks_of_interest.iterrows():
        ticker = r['ticker']
        price = r['price']
        prev_price = df_prev[df_prev['ticker'] == ticker]['price'].iloc[0] if df_prev is not None else 'NA'
        res.append((ticker, prev_price, price))

    for missing_ticker in missing_stocks:
        prev_price = df_prev[df_prev['ticker'] == missing_ticker]['price'].iloc[0] if df_prev is not None else 'NA'
        price = get_last_available_price(df_full, missing_ticker)
        res.append((missing_ticker, prev_price, price))
        
    res.sort(key = lambda t: t[0])
    return res

# ASSUMPTION: df is filtered by date.
def get_share_allocation(
    df: pd.DataFrame,
    tickers: List[str], # portfolio
    investment_amount: int,
    weight_approach: StockBasketWeightApproach
) -> List[ShareAllocation]:
    if weight_approach != StockBasketWeightApproach.EQUAL_WEIGHTING:
        raise Exception(f'{weight_approach} not supported yet.')

    amount_per_stock = investment_amount / len(tickers)
    res = []
    for ticker in tickers:
        price = df[df['ticker'] == ticker]['price'].iloc[0]
        num_shares = amount_per_stock / price
        res.append(ShareAllocation(ticker, num_shares))
    return res    
    
# ASSUMPTION: df is filtered by date.
def get_portfolio_value(
    df: pd.DataFrame,
    share_allocation: List[ShareAllocation]
) -> float:
    total = 0
    for allocation in share_allocation:
        price = df[df.ticker == allocation.ticker]['price'].iloc[0]
        total += (price * allocation.num_shares)
    return total

# ASSUMPTION: df is filtered by date.
def get_stock_basket_price(
    df: pd.DataFrame,
    df_full: pd.DataFrame, # Used to get the prices for stocks that closed
    share_allocation: List[ShareAllocation],
    should_print = False
) -> Tuple[float, Set[str]]:
    tickers = [alloc.ticker for alloc in share_allocation]
    stocks_of_interest = df.loc[df['ticker'].isin(tickers)]
    missing_stocks = set(tickers) - set(stocks_of_interest['ticker'])
    basket_price = 0
    for alloc in share_allocation:
        ticker = alloc.ticker
        if ticker in missing_stocks:
            price = get_last_available_price(df_full, ticker)
        else:
            price = df[df.ticker == ticker]['price'].iloc[0]
        basket_price += (price * alloc.num_shares)
    return (round(basket_price, 2), missing_stocks)

# get_stock_price_per_stock(base_sorted_df, prev_base_sorted_df, date_sorted_daily_data, base_portfolio)
# base_sorted_df[base_sorted_df['ticker'] == 'CVVT']
# prev_base_sorted_df[prev_base_sorted_df['ticker'] == 'CVVT']
# date_sorted_daily_data[date_sorted_daily_data['ticker'] == 'CVVT']
# get_last_available_price(date_sorted_daily_data, 'CVVT')

# share_allocation = get_share_allocation(daily_data_df, base_portfolio, 1000, StockBasketWeightApproach.EQUAL_WEIGHTING)

# print(get_portfolio_value(daily_data_df, share_allocation))
# print(get_portfolio_value(prev_daily_data_df, share_allocation))
# print(get_stock_basket_price_by_share_allocation(daily_data_df, date_sorted_daily_data, share_allocation))
# print(get_stock_basket_price_by_share_allocation(prev_daily_data_df, date_sorted_daily_data, share_allocation))

In [6]:
# Prepare Inputs for Base + Test
INITIAL_PORTFOLIO_VALUE = 10000
PORTFOLIO_SIZE = 30
REBALANCE_DAYS = 90

BASE_METRIC = EvaluationMetric.EV_EBIT
TEST_METRIC = EvaluationMetric.P_B
STOCKS_UNIVERSE = StockUniverse.LARGE
PORTFOLIO_WEIGHT_STRATEGY = StockBasketWeightApproach.EQUAL_WEIGHTING

# OPTIMIZATION ONLY: Sorting every time would take too long...
date_sorted_daily_data = daily_data.sort_values(by='date')
base_sorted_daily_data = sort_df_by_metric(daily_data, BASE_METRIC)
test_sorted_daily_data = sort_df_by_metric(daily_data, TEST_METRIC)

In [9]:
start_date = datetime.datetime.strptime(min(daily_data['date']), '%Y-%m-%d')
end_date = datetime.datetime.strptime(max(daily_data['date']), '%Y-%m-%d')
        
rebalance_dates = get_rebalance_dates(start_date, end_date, timedelta(days=REBALANCE_DAYS))

base_portfolio_value = INITIAL_PORTFOLIO_VALUE
test_portfolio_value = INITIAL_PORTFOLIO_VALUE
portfolio_size = PORTFOLIO_SIZE

start_date = rebalance_dates[0]

# Get data for start date
daily_data_df = filter_df_by_date(date_sorted_daily_data, start_date)
base_sorted_df = filter_df_by_date(base_sorted_daily_data, start_date)
test_sorted_df = filter_df_by_date(test_sorted_daily_data, start_date)

# Filter based on the universe of stocks we are interested in.
base_sorted_df_in_universe = filter_stocks_by_universe(base_sorted_df, STOCKS_UNIVERSE)
test_sorted_df_in_universe = filter_stocks_by_universe(test_sorted_df, STOCKS_UNIVERSE)

# Get base stocks from the universe selected for each metric
base_portfolio = get_top_n_stocks_by_metric(base_sorted_df_in_universe, portfolio_size, BASE_METRIC)
test_portfolio = get_top_n_stocks_by_metric(test_sorted_df_in_universe, portfolio_size, TEST_METRIC)

# Compute number of shares we can buy of each stock
base_share_allocation = get_share_allocation(daily_data_df, base_portfolio, base_portfolio_value, PORTFOLIO_WEIGHT_STRATEGY)
test_share_allocation = get_share_allocation(daily_data_df, test_portfolio, test_portfolio_value, PORTFOLIO_WEIGHT_STRATEGY)

# SANITY CHECK: compute these values rather than assigning them for consistency.
base_price, base_tickers_closed = get_stock_basket_price(base_sorted_df, date_sorted_daily_data, base_share_allocation)
test_price, test_tickers_closed = get_stock_basket_price(test_sorted_df, date_sorted_daily_data, test_share_allocation)

# SANITY CHECK
assert INITIAL_PORTFOLIO_VALUE == base_price
assert INITIAL_PORTFOLIO_VALUE == test_price
assert base_portfolio_value == base_price
assert test_portfolio_value == test_price

res = {}
debug = {}

res[start_date] = {
    'base_price': base_price,
    'test_price': test_price,
}

prev_base_price = base_price
prev_test_price = test_price
prev_date = start_date

# Skip the first date (start_date) because it is handeled above
for date in rebalance_dates[1:]:
    print(date.strftime('%Y-%m-%d'))

    # Filter DFs for optimization purposes
    daily_data_df = filter_df_by_date(date_sorted_daily_data, date)
    base_sorted_df = filter_df_by_date(base_sorted_daily_data, date)
    test_sorted_df = filter_df_by_date(test_sorted_daily_data, date)

    # Compute value of previous portfolio at today's date
    base_price, base_tickers_closed = get_stock_basket_price(base_sorted_df, date_sorted_daily_data, base_share_allocation)
    test_price, test_tickers_closed = get_stock_basket_price(test_sorted_df, date_sorted_daily_data, test_share_allocation)

    # Compute teh change in the portfolio value
    base_change = base_price / prev_base_price
    test_change = test_price / prev_test_price

    # Compute new portfolio value
    base_portfolio_value = round(base_portfolio_value * base_change, 2)
    test_portfolio_value = round(test_portfolio_value * test_change, 2)    

    res[date] = {
        'base_price_prev': prev_base_price,
        'base_price': base_price,
        'test_price_prev': prev_test_price,        
        'test_price': test_price,      
    }

    # Get data for the previous date to compute `base_portfolio_per_ticker_change` for debugging below
    prev_daily_data_df = filter_df_by_date(date_sorted_daily_data, prev_date)
    prev_base_sorted_df = filter_df_by_date(base_sorted_daily_data, prev_date)
    prev_test_sorted_df = filter_df_by_date(test_sorted_daily_data, prev_date)        
    
    # DEBUG ONLY
    debug[date] = {
        'prev_date': prev_date,
        'curr_date': date,
        'base_portfolio_prev_price': prev_base_price,
        'base_portfolio_curr_price': base_price,
        'base_portfolio_tickers_closed': base_tickers_closed,
        'base_portfolio_per_ticker_data': get_stock_price_per_stock(base_sorted_df, prev_base_sorted_df, date_sorted_daily_data, base_portfolio),                
    }
    
    # Filter based on the universe of stocks we are interested in.
    base_sorted_df_in_universe = filter_stocks_by_universe(base_sorted_df, STOCKS_UNIVERSE)
    test_sorted_df_in_universe = filter_stocks_by_universe(test_sorted_df, STOCKS_UNIVERSE)    

    # Get the newley selected portfolio.
    base_portfolio = get_top_n_stocks_by_metric(base_sorted_df_in_universe, portfolio_size, BASE_METRIC)
    test_portfolio = get_top_n_stocks_by_metric(test_sorted_df_in_universe, portfolio_size, TEST_METRIC)

    base_share_allocation = get_share_allocation(daily_data_df, base_portfolio, base_portfolio_value, PORTFOLIO_WEIGHT_STRATEGY)
    test_share_allocation = get_share_allocation(daily_data_df, test_portfolio, test_portfolio_value, PORTFOLIO_WEIGHT_STRATEGY)    
    
    # Get new portfolio price    
    base_price, base_tickers_closed = get_stock_basket_price(base_sorted_df, date_sorted_daily_data, base_share_allocation)
    test_price, test_tickers_closed = get_stock_basket_price(test_sorted_df, date_sorted_daily_data, test_share_allocation)    
    
    # SANITY CHECK: since we just got these stocks, none of them should be closed...
    assert len(base_tickers_closed) == 0
    assert len(test_tickers_closed) == 0
    
    # Cache the last price of the current portfolio (previous date)
    prev_base_price = base_price
    prev_test_price = test_price
    prev_date = date
    
    # DEBUG ONLY: Adding this to the debug DF for easier debugging...
    debug[date].update({
        'new_base_portfolio_per_ticker_data': get_stock_price_per_stock(base_sorted_df, None, date_sorted_daily_data, base_portfolio),
    })
    
df_res = pd.DataFrame.from_dict(res, orient='index')
df_debug = pd.DataFrame.from_dict(debug, orient='index')    

2011-04-01
2011-07-01
2011-09-30
2011-12-29
2012-03-28
2012-06-26
2012-09-24
2012-12-21
2013-03-22
2013-06-21
2013-09-19
2013-12-18
2014-03-18
2014-06-16
2014-09-12
2014-12-12
2015-03-13
2015-06-11
2015-09-09
2015-12-08
2016-03-07
2016-06-03
2016-09-02
2016-12-02
2017-03-02
2017-05-31
2017-08-29
2017-11-27
2018-02-23
2018-05-25
2018-08-24
2018-11-21
2019-02-20
2019-05-21
2019-08-19
2019-11-15
2020-02-14
2020-05-15
2020-08-13
2020-11-10
2021-02-09
2021-05-10


In [10]:
write_df_debug_to_feather(df_debug, REBALANCE_DAYS)
write_df_res_to_feather(df_res, REBALANCE_DAYS)

In [12]:
df_debug.iloc[0:2]

,prev_date,curr_date,base_portfolio_prev_price,base_portfolio_curr_price,base_portfolio_tickers_closed,base_portfolio_per_ticker_data,new_base_portfolio_per_ticker_data
2011-04-01,2011-01-03,2011-04-01,10000.00,9043.29,{},"[(ADGI, 3.3, 3.52), (AXGN, 4.0, 2.84), (BDSI, 3.55, 3.55), (BOPH, 1.79, 1.7), (CBFV, 12.056, 12.329), (CHTR, 38.46, 51.66), (CSKI, 6.93, 3.46), (DYNP, 2.9, 1.15), (ESGR, 86.76, 99.84), (FBSS, 9.465, 10.21), (FUQI, 6.29, 3.0), (GAI, 8.125, 6.047), (GDEN, 5.98, 5.38), (GEVA, 12.75, 12.35), (GRVY, 6.72, 7.96), (HWG, 26.2, 27.0), (IPSU, 13.202, 13.306), (LUNA, 1.7, 2.1), (MILL, 5.35, 5.14), (NTIP, 1.357, 1.357), (OPTI, 0.693, 0.62), (PTP, 43.987, 37.408), (RINO, 3.9, 1.85), (SPMD1, 10.13, 6.21), (SSFC, 4.45, 4.7), (TCCO, 12.282, 8.965), (TGS, 2.581, 2.166), (TKPPY, 23.725, 26.8), (VABK, 11.693, 11.423), (VIVHY, 18.516, 15.545)]","[(ABAT, NA, 2.07), (ADGI, NA, 3.52), (ATV, NA, 17.662), (BOPH, NA, 1.7), (CAGM, NA, 0.66), (CBEH, NA, 2.28), (CEXE, NA, 0.4), (CHBT, NA, 8.38), (CMFB, NA, 9.0), (CNEP, NA, 4.79), (CSKI, NA, 3.46), (ESGR, NA, 99.84), (FMBH, NA, 14.944), (GEVA, NA, 12.35), (GRVY, NA, 7.96), (HWG, NA, 27.0), (MAXY, NA, 2.238), (NBTF, NA, 15.997), (NTIP, NA, 1.357), (OPTI, NA, 0.62), (PRSG, NA, 0.14), (RINO, NA, 1.85), (RIVE, NA, 8.006), (SCEI, NA, 4.91), (SOMH, NA, 8.228), (SSFC, NA, 4.7), (TCCO, NA, 8.965), (TGS, NA, 2.166), (UTRA, NA, 4.19), (VLOV, NA, 13.8)]"
2011-07-01,2011-04-01,2011-07-01,9043.29,7401.58,{},"[(ABAT, 2.07, 1.07), (ADGI, 3.52, 3.5), (ATV, 17.662, 19.914), (BOPH, 1.7, 0.91), (CAGM, 0.66, 0.6), (CBEH, 2.28, 0.83), (CEXE, 0.4, 0.15), (CHBT, 8.38, 1.8), (CMFB, 9.0, 9.35), (CNEP, 4.79, 3.35), (CSKI, 3.46, 2.16), (ESGR, 99.84, 106.04), (FMBH, 14.944, 14.571), (GEVA, 12.35, 12.5), (GRVY, 7.96, 6.76), (HWG, 27.0, 19.01), (MAXY, 2.238, 2.341), (NBTF, 15.997, 16.885), (NTIP, 1.357, 1.247), (OPTI, 0.62, 0.562), (PRSG, 0.14, 0.149), (RINO, 1.85, 0.54), (RIVE, 8.006, 9.213), (SCEI, 4.91, 1.34), (SOMH, 8.228, 8.051), (SSFC, 4.7, 5.5), (TCCO, 8.965, 7.777), (TGS, 2.166, 2.432), (UTRA, 4.19, 3.96), (VLOV, 13.8, 10.5)]","[(ADGI, NA, 3.5), (BOPH, NA, 0.91), (BSPM, NA, 24.15), (CAGM, NA, 0.6), (CETC, NA, 47.8), (CMFB, NA, 9.35), (CNEP, NA, 3.35), (CNET, NA, 3.675), (CNGL, NA, 1.85), (CSPI, NA, 2.773), (ELLO, NA, 6.914), (FTFT, NA, 21.44), (GEVA, NA, 12.5), (GIGM, NA, 6.5), (GPRC, NA, 2.6), (GRVY, NA, 6.76), (GURE, NA, 16.0), (KZ, NA, 3.836), (LPHIQ, NA, 2.443), (LPIH, NA, 1.49), (MAXY, NA, 2.341), (MCBC, NA, 2.398), (NEWN, NA, 2.14), (NTIP, NA, 1.247), (PRSG, NA, 0.149), (SEII, NA, 1880.0), (TCCO, NA, 7.777), (TGS, NA, 2.432), (TPI, NA, 1.47), (VRNM, NA, 1.72)]"


In [13]:
df_res.iloc[0:2]

,base_price,test_price
2011-01-03,10000.00,10000.00
2011-04-01,9043.29,10288.22
